In [ ]:
from hypster import HypsterX

# Basic

In [ ]:
data = ...

In [ ]:
transformations = [?]

In [ ]:
#Q: where should we put the data? in HyPSTER or at ".fit"?

In [ ]:
model = HypsterX(data)

In [ ]:
model.fit(n_trials=100)

In [ ]:
#Q: how to handle "predict" with FastAI stuff?
model.predict(X_test)

# Advanced

In [ ]:
from hypster.constraints import *

In [4]:
from hypster.samplers import CmaEsSampler
from hypster.callbacks.pruners import SuccessiveHalvingPruner
from hypster.callbacks import EarlyStopping, ReduceLROnPlateau

## Define X

In [ ]:
constraints = [CPUOnly, InterpretableFeatures, 
               ExplainableModel, FastTraining]

In [ ]:
models = [?]

In [ ]:
#Q: How should the user specify models & frameworks

In [ ]:
#Q: How should we handle negation, i.e. No GPU (?)

In [ ]:
#exclude = [GPU, NeuralNetworks] (?)

In [ ]:
sampler = CmaEsSampler()

### Common Callbacks

In [ ]:
pruner = SuccessiveHalvingPruner()

In [ ]:
early_stopper = EarlyStopping(tol=1e-5, patience=5)

In [ ]:
lr_reducer = ReduceLROnPlateau(lr_decay=0.1)

### Specific Callbacks

In [ ]:
#Q: How to specify which framework this callbacks goes to?
#Q: How to avoid collisions in Callback names between packages?
fastai_cbs = FastAICallback([BatchLossFilter, fp16, ...])

In [ ]:
cbs = [pruner, early_stopper, lr_reducer, fastai_cbs]

## Model

In [ ]:
model = HyPSTERTabular(data, constraints, sampler, cbs=cbs)

In [ ]:
model.fit(n_trials=100)

In [ ]:
model.predict_proba(X_test)

# Advanced #2

In [ ]:
from hypster.hps import *
from hypster.distributions import *

### Common HPs

In [ ]:
#tree_depth, learning_rate, optimizers, regularization, ...

In [ ]:
tree_depth = TreeDepthHP(min=2, max=6, dist=UniformIntDistribution) 

In [ ]:
#Q: How to handle LR with distribution or LRFinder?
lr = LearningRateHP(start_lr, end_lr, dist(?)=LRFinder(...))

### Specific HPs

In [ ]:
xgboost_l1_reg = LogUniformHP(model=xgboost, 
                              hp_name="l1_reg", 
                              start_value=0.1, end_value=0.5,
                              default=0.1)

# Build Model

In [ ]:
hps = [lr, tree_depth, xgboost_l1_reg]

In [ ]:
model = HyPSTERX(hp_override=hps)

In [ ]:
model.fit(data, n_trials=100)

In [ ]:
model.predict(X_test)

# FastAI Example

In [ ]:
df = pd.read_csv(path/'adult.csv')

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df))

In [ ]:
to = TabularPandas(df, procs, cat_names, cont_names, y_names="salary", splits=splits)

In [ ]:
trn_dl = TabDataLoader(to.train, bs=64, num_workers=0, shuffle=True, drop_last=True)
val_dl = TabDataLoader(to.valid, bs=128, num_workers=0)
dbunch = DataBunch(trn_dl, val_dl)
dbunch.show_batch()

In [ ]:
model = TabularModel(get_emb_sz(to), len(to.cont_names), 2, [200,100])

In [ ]:
opt_func = partial(Adam, wd=0.01, eps=1e-5)
learn = Learner(dbunch, model, CrossEntropyLossFlat(), opt_func=opt_func, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(1)